# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [20]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Question 1
q1_generator = square_root_generator(5)
print(sum([sqrt_value for sqrt_value in q1_generator]))


# Question 2
q2_generator = square_root_generator(13)
for index, sqrt_value in enumerate(q2_generator):
    if index + 1 == 13:
      print(sqrt_value)

8.382332347441762
3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [21]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

people_1_table = [person for person in people_1()]
# print(sum([person['Age'] for person in people_1_table]))


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


people_2_table = [person for person in people_2()]
combined = people_1_table + people_2_table
print(sum([people['Age'] for people in combined]))


353


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [12]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [13]:
# to do: homework :)

import dlt

generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')

run1 = generators_pipeline.run(people_1(), table_name='people', write_disposition='replace', primary_key='ID')

run2 = generators_pipeline.run(people_2(), table_name='people', write_disposition='merge', primary_key='ID')

print(run2)

Pipeline dlt_colab_kernel_launcher load step completed in 0.66 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707805771.339212 is LOADED and contains no failed jobs


In [18]:
import duckdb

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))
people = conn.sql("select * from people").df()
display(people)

total_age = conn.sql("select sum(age) from people").df()
display(total_age)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
└─────────────────────┘

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707805770.4610991,EJvsnOHo849l3g,None
1,2,Person_2,27,City_A,1707805770.4610991,Bf6NbttWU6CQnw,None
2,8,Person_8,38,City_B,1707805771.339212,gYd1sKyCVYlTgQ,Job_8
3,4,Person_4,34,City_B,1707805771.339212,nYDZh6BFw1cTRw,Job_4
4,5,Person_5,35,City_B,1707805771.339212,supKF4q3s8DH5w,Job_5
5,7,Person_7,37,City_B,1707805771.339212,uoY+j50oqcouZQ,Job_7
6,3,Person_3,33,City_B,1707805771.339212,yNlCGo7HGkKHdw,Job_3
7,6,Person_6,36,City_B,1707805771.339212,I1Utj8jbJSsqUg,Job_6


,sum(age)
0,266.0


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX